In [ ]:
# importing neccesary libraries and modules

import os
import requests
import openai
from openai import AzureOpenAI
from typing import Annotated

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt

# API keys and such. Since I'm pretty sure I'm not supposed to share the expired keys I'll leave them out. 

endpoint = "///"
model_name = "///"
deployment = "///"

subscription_key = "///"
api_version = "///"

# Initializing the cilent

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

# Human input handling

def human_rep(query: str) -> str:
    human_response = interrupt({"query" : query})
    return human_response["data"]

# Creating a list for the messages

class State(TypedDict):
    messages: Annotated[list, add_messages]



# Defining the chatbot and its responses and behavior

def chatbot(state: State):
    last_message = state["messages"][-1]
    messages = []
    
    messages.append({
        "role": "system",
        "content": "You are a freight carrier negotiating the price for a load. Always negotiate above target price for profit!"
                     
                     })
    # Checks if message has content and then checks if it came from user or system

    for msg in state["messages"]:
        if hasattr(msg, "content"):
            role = "user" if hasattr(msg, 'type') and msg.type == "human" else "assistant"
            messages.append({
                "role" : role,
                "content" : msg.content


            })
        else:
            # adds message
            messages.append(msg)
    
    response = client.chat.completions.create(
        # deploys model, grabs messages from above list, sets max tokens

        model=deployment,
        messages = messages,
        max_completion_tokens=16384,
    )
    
    # Returns response from chatbot

    chatbot_rep = (response.choices[0].message.content)
    return {"messages": [{"role": "assistant", "content": chatbot_rep}]}


# Builds the graph and start/end nodes

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Saves and compiles memory and the graph

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Takes user input

user_input = input("Hello, how can I help you today? ")
config = {"configurable" : {"thread_id" : "1"}}

print("\n~INITIAL REPONSE~")

# Takes the reponse and prints it out 

events = graph.stream(
    {"messages" : [{"role" : "user", "content": user_input}]},
    config,
    stream_mode = "values",
    )


for event in events:
    if "messages" in event:
        last_message = event["messages"][-1]
        if hasattr(last_message, 'pretty_print'):
            last_message.pretty_print()
        else:
            print(f"{last_message.get('role', 'unknown')}: {last_message.get('content', '')}")

# Loops until the user says "bye"

running = True
while running:
    user_input = input("You: ")
    if user_input == "bye":
        running = False
        break
    
    # New reponses in the loop
    
    events = graph.stream(
    {"messages" : [{"role" : "user", "content": user_input}]},
    config,
    stream_mode = "values",
    )

    for event in events:
        if "messages" in event:
            last_message = event["messages"][-1]
            if hasattr(last_message, 'pretty_print'):
                last_message.pretty_print()
            else:
                print(f"{last_message.get('role', 'unknown')}: {last_message.get('content', '')}")

            










~INITIAL REPONSE~
================================ Human Message =================================

Dallas, TX → Los Angeles, CA Commodity: Electronics (dry van) Weight: 30,000 lbs Distance: ~1,450 miles Target Price: $2,250
================================== Ai Message ==================================

Thanks — happy to move this. Based on Dallas → Los Angeles, ~1,450 miles, electronics, 30,000 lbs (assume standard palletized dry-van, dock-to-dock, single stop):

My all-in asking rate: $3,150 ($2.17/mile; ≈ $0.105/lb).  
If you can confirm and book today I can drop to $2,950. My bottom acceptable: $2,700.

What this assumes
- Standard dock-to-dock pickup & delivery, single stop
- Palletized, shrink-wrapped, no special equipment or hazmat
- No detention, lumper, liftgate, inside delivery, or extra stops
- Normal insurance/declared value for electronics (if higher declared value required, rate adjusts)

If any of the above apply (liftgate, residential/inside delivery, appointments, e